In [1]:
from pyathena import connect
import pandas as pd
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('../access_keys_shreya.cfg')
AWS_ACCESS_KEY = config.get('aws', 'aws_access_key')
AWS_SECRET_KEY = config.get('aws', 'aws_secret_key')

In [3]:
from six.moves.urllib.parse import quote_plus
from sqlalchemy.engine import create_engine

SCHEMA_NAME = "yelp"
S3_STAGING_DIR = "s3://sairin.yelp.dataset/dataset"
AWS_REGION = "us-west-1"
conn_str = (
    "awsathena+rest://{aws_access_key_id}:{aws_secret_access_key}@"
    "athena.{region_name}.amazonaws.com:443/"
    "{schema_name}?s3_staging_dir={s3_staging_dir}&work_group=primary"
)


# Create the SQLAlchemy connection. Note that you need to have pyathena installed for this.
engine = create_engine(
    conn_str.format(
        aws_access_key_id=quote_plus(AWS_ACCESS_KEY),
        aws_secret_access_key=quote_plus(AWS_SECRET_KEY),
        region_name=AWS_REGION,
        schema_name=SCHEMA_NAME,
        s3_staging_dir=quote_plus(S3_STAGING_DIR),
    )
)

conn = engine.connect()


In [4]:
# pyathena connection
conn = connect(
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    s3_staging_dir=S3_STAGING_DIR,
    region_name=AWS_REGION,
    schema_name=SCHEMA_NAME,  # Optional: Specify the schema name
    
)

In [5]:
query = 'SHOW TABLES IN yelp'
tables = pd.read_sql(query, conn)

tables

/var/folders/69/1v36mqw920lcphv2lfprw2sw0000gp/T/ipykernel_88478/2741500285.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql(query, conn)


,tab_name
0,balanced_reviews_dataset
1,businesses
2,checkins
3,philadelphia_top_restaurant
4,reviews
5,reviews_of_top_restaurants
6,tips
7,top_fifty_visited_restaurant
8,twenty_most_frequent_restaurant
9,users


In [6]:
query_business = """
        SELECT * 
        FROM yelp.businesses limit 10
        """

df = pd.read_sql(query_business, conn)

df.head()


/var/folders/69/1v36mqw920lcphv2lfprw2sw0000gp/T/ipykernel_88478/44002146.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_business, conn)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{byappointmentonly=True},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{businessacceptscreditcards=True},"Shipping Centers, Local Services, Notaries, Ma...","{""saturday"":""8:0-14:0"",""tuesday"":""8:0-18:30"",""..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{businessacceptscreditcards=True, bikeparking=...","Department Stores, Shopping, Fashion, Home & G...","{""sunday"":""8:0-22:0"",""saturday"":""8:0-23:0"",""tu..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{restaurantspricerange2=1, restaurantstakeout=...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{""sunday"":""7:0-21:0"",""saturday"":""7:0-21:0"",""tu..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{restaurantstakeout=True, businessacceptscredi...","Brewpubs, Breweries, Food","{""sunday"":""12:0-18:0"",""saturday"":""12:0-22:0"",""..."


In [15]:
df.to_csv("business_table.csv", index=False )

In [20]:
import geocoder

# Obtain GPS coordinates based on address or location
g = geocoder.osm("Portland, NM")
q_lat = g.lat
q_lon = g.lng
q = (q_lat, q_lon)
print("Latitude:", q_lat)
print("Longitude:", q_lon)

Latitude: 32.2918956
Longitude: -106.802413


In [14]:
from geopy.distance import geodesic

In [21]:
def distance_to_q(coord1, q=q):
    return geodesic(coord1, q).miles

In [23]:
df_sample = df.sample(100)

In [25]:
df.apply(lambda row: distance_to_q((row.latitude, row.longitude), q), axis=1).sort_values

65448     1267.773896
8567       239.110778
147131    1840.158190
15569     1043.752928
47273     1468.622582
             ...     
135358    1258.666025
28306     1476.085438
105758    1478.180117
91606     1467.244211
50738     1479.001655
Length: 100, dtype: float64

In [75]:
def search_by_location_and_cuisine_type(location, cuisine_type="", n_results=20, df=df):
    coordinates = geocoder.osm(location)
    q = (coordinates.lat, coordinates.lng)
    print(q)
    cuisine_filter = df["categories"].str.split(", ").explode().str.lower().str.contains(cuisine_type.lower()).dropna()
    df_filtered = df.loc[cuisine_filter[cuisine_filter].index]
    print("filtered")
    distances = df_filtered.apply(lambda row: distance_to_q((row.latitude, row.longitude), q), axis=1).sort_values()
    result = df_filtered.loc[distances.index].head(n_results)
    result["distance"] = distances.head(n_results)
    return result, distances, coordinates
    

In [83]:
res, d, g= search_by_location_and_cuisine_type("Philadelphia, PA", "Mexican")
res

(39.9527237, -75.1635262)
filtered


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,distance
89795,0ZQVSAC4uwZR1JXmDrh9CQ,Taco Bell,2 Penn Center Plz,Philadelphia,PA,19102,39.952955,-75.165476,2.5,6,0,"{businessacceptscreditcards=True, restaurantsr...","Tex-Mex, Restaurants, Mexican, Fast Food",None,0.104765
69903,S21Myaj831l5rG4AKEyekA,Dos Tacos,120 S 15th St,Philadelphia,PA,19102,39.950572,-75.165793,3.0,258,1,"{alcohol=u'none', businessacceptscreditcards=T...","Mexican, Restaurants","{""saturday"":""11:0-15:0"",""tuesday"":""11:0-21:0"",...",0.191126
132958,6F-6mFFNw2YXN7lD_Gukcg,Lolita,106 S 13th St,Philadelphia,PA,19107,39.950081,-75.162148,4.0,423,0,"{alcohol=u'full_bar', businessacceptscreditcar...","Restaurants, Mexican","{""sunday"":""17:0-0:0"",""saturday"":""17:0-0:0"",""tu...",0.196466
37555,85e8Uyn8tIyEZCqBP80rqA,Lolita,106 S 13th St,Philadelphia,PA,19107,39.950081,-75.162000,4.0,668,1,"{alcohol=u'full_bar', businessacceptscreditcar...","Restaurants, Mexican","{""sunday"":""16:0-21:0"",""saturday"":""12:0-22:0"",""...",0.199535
146673,n38QafGVzbdjavNHtCcpqA,12th Street Cantina,Reading Terminal Market,Philadelphia,PA,19107,39.953296,-75.159439,3.0,83,0,"{alcohol=u'none', businessacceptscreditcards=T...","Restaurants, Mexican","{""sunday"":""8:0-18:0"",""saturday"":""8:0-18:0"",""tu...",0.220585
74809,9PZxjhTIU7OgPIzuGi89Ew,El Vez,121 S 13th St,Philadelphia,PA,19107,39.949702,-75.161770,4.0,3187,1,"{restaurantsreservations=True, goodforkids=Tru...","Lounges, Bars, Nightlife, Breakfast & Brunch, ...","{""sunday"":""12:0-22:0"",""saturday"":""16:0-22:0"",""...",0.228376
122648,G9LZoNlCfRH941q87_JLIg,Mission Taqueria,"1516 Sansom St, Fl 2",Philadelphia,PA,19102,39.950222,-75.166553,4.0,617,1,"{alcohol=u'full_bar', businessacceptscreditcar...","Tex-Mex, Restaurants, Food, Nightlife, Beer, W...","{""saturday"":""11:0-23:0"",""tuesday"":""11:0-22:0"",...",0.235857
145620,o4gT7fMheja86ur9PVb2Iw,District Taco,1140 Market St,Philadelphia,PA,19107,39.951649,-75.159148,3.0,74,1,"{restaurantsreservations=False, goodforkids=Tr...","Restaurants, Mexican","{""sunday"":""10:0-17:0"",""saturday"":""10:0-21:0"",""...",0.244007
28682,4rhH2r1lL7ujqnvTwZo--w,Chipotle Mexican Grill,"1625 Chestnut St, Ste F6",Philadelphia,PA,19103,39.951571,-75.168004,3.5,24,1,"{alcohol=u'none', businessacceptscreditcards=T...","Restaurants, Mexican, Fast Food","{""sunday"":""12:0-18:0"",""saturday"":""11:0-19:0"",""...",0.250712
71491,DvyG9Fx1EeNJfr_UNL7NYA,Fresco Tortilla,200 S Broad St Hyatt Food Ct,Philadelphia,PA,19102,39.949224,-75.164942,3.5,10,0,"{alcohol=u'none', businessacceptscreditcards=T...","Event Planning & Services, Restaurants, Mexica...","{""tuesday"":""9:0-17:0"",""wednesday"":""9:0-17:0"",""...",0.252888


In [ ]:
res